In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
train_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Training & Testing Spliting
training_set = train_datagen.flow_from_directory(r'dataset1/train',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')


In [ ]:
test_set = train_datagen.flow_from_directory(r'dataset1/validation',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

In [ ]:
inception_v3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in inception_v3.layers[: -15]:
    layer.trainable = False

In [ ]:
x = inception_v3.output
x = Flatten()(x)

x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(units=7, activation='softmax')(x)
model = Model(inception_v3.input, output)

#model.summary()

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
histry = model.fit(training_set,
                            validation_data=test_set,
                            epochs=100,
                            steps_per_epoch=len(training_set),
                            validation_steps=len(test_set),
                            )

In [ ]:
plt.plot(histry.history['loss'], label='train loss')
plt.legend()
plt.show()

# plot the accuracy
plt.plot(histry.history['accuracy'], label='accuracy')
plt.legend()
plt.show()


model.save('autism_model.h5')